In [29]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread, imshow
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xml.etree.ElementTree as ET

In [49]:
img_dir = "images/Images"
ant_dir = "annotations/Annotation"
X=[] #input array
Y=[] #output array

bnd_dict = {}

label_count = 0
for sub_dir in os.listdir(ant_dir):
    label_count += 1
    label = sub_dir.split("-")[1]
    print(label)
    path=os.path.join(ant_dir,sub_dir)
    for ant in os.listdir(path):
        if ant[0] != 'n': continue
        tree = ET.parse(os.path.join(path,ant))
        root = tree.getroot()
        bndbox = root.find("object").find("bndbox")
        xmin = int(bndbox.findtext("xmin"))
        ymin = int(bndbox.findtext("ymin"))
        xmax = int(bndbox.findtext("xmax"))
        ymax = int(bndbox.findtext("ymax"))
        bnd_dict[ant] = [xmin, ymin, xmax, ymax]
    if label_count >= 3: break
#print(bnd_dict)
        
        
        
label_count = 0
for sub_dir in os.listdir(img_dir):
    label_count += 1
    label = sub_dir.split("-")[1]
    print(label)
    path=os.path.join(img_dir,sub_dir)
    for img in os.listdir(path):
        #print(img)
        if img[0] != 'n': continue
        img_array=imread(os.path.join(path,img))
        #imshow(img_array)
        #plt.show()
        bnd = bnd_dict[img.split(".")[0]]
        img_bnd = img_array[bnd[1]:bnd[3],bnd[0]:bnd[2]]
        #imshow(img_bnd)
        #plt.show()
        
        img_resized=resize(img_bnd,(150,150,3))
        
        X.append(img_resized.flatten())
        Y.append(label_count-1)
    print(f'loaded category:{label} successfully')
    if label_count >= 3: break

X=np.array(X)
Y=np.array(Y)
print(X)
print(Y)

Chihuahua
Japanese_spaniel
Maltese_dog
Chihuahua
loaded category:Chihuahua successfully
Japanese_spaniel
loaded category:Japanese_spaniel successfully
Maltese_dog
loaded category:Maltese_dog successfully
[[0.24933063 0.1905071  0.1787424  ... 0.01027991 0.02237142 0.01894658]
 [0.82930736 0.17205229 0.34640505 ... 0.52018231 0.62311041 0.4026658 ]
 [0.55686275 0.54509804 0.54901961 ... 0.67840078 0.6889651  0.53602392]
 ...
 [0.31753926 0.30475817 0.27338562 ... 0.15334963 0.14901961 0.12941176]
 [0.42766013 0.44334641 0.44726797 ... 0.67705882 0.55917647 0.42420889]
 [0.57516209 0.53202484 0.51633856 ... 0.46044706 0.42515294 0.38985882]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [50]:
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid)

In [51]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=77,stratify=Y)
print('Splitted Successfully')
model.fit(x_train,y_train)
print('The Model is trained well with the given images')

Splitted Successfully
The Model is trained well with the given images


In [52]:
y_pred=model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

The predicted Data is :
[2 1 2 0 0 0 2 1 2 2 0 0 0 2 0 2 1 2 2 2 2 2 0 2 2 0 2 2 1 2 1 1 2 0 2 1 0
 1 1 2 1 0 2 2 2 2 1 1 2 2 1 1 1 1 1 2 2 0 2 1 0 1 0 0 0 2 2 0 1 2 1 1 2 1
 1 0 2 2 2 2 0 1 2 2 2 2 1 1 1 2 2 1 2 2 2 2 2 2 1 0 0 2 0 1 2 2 2 2 0 2 0
 0 0 0 2 0 1 2]
The actual data is:
[0 2 2 1 0 0 2 1 1 2 0 0 0 2 1 2 2 2 2 2 2 2 0 0 1 0 2 2 1 2 1 0 2 1 2 1 0
 1 0 2 1 1 2 2 2 1 0 1 2 2 1 1 1 1 1 2 2 2 1 1 1 1 2 0 0 2 2 0 1 2 1 1 2 2
 1 0 2 2 2 2 0 0 2 2 2 1 1 1 1 2 1 1 2 2 2 2 0 0 0 0 0 2 2 1 0 0 2 0 1 2 2
 0 0 1 1 0 2 2]
The model is 71.1864406779661% accurate
